In [3]:
import os
import shutil
import re
import subprocess
import urllib
import zipfile
import requests


"""
Scrapes and installs chromium from linux mint 21.3(virginia) packages site.
Link: http://packages.linuxmint.com/pool/upstream/c/chromium/
Scrapes and installs chromedriver from Chrome for Testing page.
Link: https://googlechromelabs.github.io/chrome-for-testing/
"""

class CantGetLatestChromiumVersionError(Exception):
    """Happens when regex failed"""

class ChromiumInstallationFailedException(Exception):
    """
    Happens when deb package not installed
    Check the downloaded chroumium deb file
    """

class CantGetChromeDriverError(Exception):
    """Happens when regex failed"""

main_url = "http://packages.linuxmint.com/pool/upstream/c/chromium/"
work_dir = "/content"

def get_chromium_latest_version() -> str:
    # A request to packages.linuxmint.com for getting latest version of chromium
    # e.g. "chromium_121.0.6167.160~linuxmint1+virginia_amd64.deb"
    r = requests.get(main_url)
    if r.status_code != 200:
        raise Exception("status_code code not 200!")
    text = r.text

    # Find latest version
    pattern = '<a\shref="(chromium_[^"]+linuxmint1%2Bvirginia_amd64.deb)'
    latest_version_search = re.search(pattern, text)
    if latest_version_search:
        latest_version = latest_version_search.group(1)
    else:
        raise CantGetLatestChromiumVersionError("Failed to get latest chromium version!")
    return latest_version

def install_chromium(latest_version: str, deb_file: str, quiet: bool):
    # Full url of deb file
    url = f"{main_url}{latest_version}"

    # Download deb file
    if quiet:
        command = f"wget -q -O {work_dir}/{deb_file} {url}"
    else:
        command = f"wget -O {work_dir}/{deb_file} {url}"
    print(f"Downloading: {deb_file}")
    # os.system(command)
    !$command

    # Install deb file
    if quiet:
        command = f"apt-get install {work_dir}/{deb_file} >> apt.log"
    else:
        command = f"apt-get install {work_dir}/{deb_file}"
    print(f"Installing: {deb_file}")
    # os.system(command)
    !$command

def check_chromium_installation(deb_file: str):
    try:
        subprocess.call(["chromium"])
        print("Chromium installation successfull.\n")
        # If installation successfull we can remove deb file
        # Delete deb file from disk
        os.remove(f"{work_dir}/{deb_file}")
    except FileNotFoundError:
        raise ChromiumInstallationFailedException("Chromium Installation Failed!")

def get_chromedriver_url(deb_file: str) -> str:
    # Get content of crhomedriver page
    url = "https://googlechromelabs.github.io/chrome-for-testing/"
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception("status_code code not 200!")
    text = r.text

    # Get chromium version from deb file's name
    version_number = deb_file.split("chromium_")[-1].split(".")[0]

    # Example: https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/121.0.6167.85/linux64/chromedriver-linux64.zip
    pattern = f'https://[^<]+/{version_number}[^<]+/linux64/chromedriver-linux64.zip'
    # Find latest version
    chromedriver_url_search = re.search(pattern, text)
    if chromedriver_url_search:
        chromedriver_url = chromedriver_url_search.group()
        return chromedriver_url
    else:
        raise CantGetChromeDriverError("Failed to get chromedriver!")

def install_chromedriver(deb_file: str, quiet: bool):
    url = get_chromedriver_url(deb_file)
    file_name = url.split("/")[-1]
    # Download chromedriver
    chromedriver_zip = f"{work_dir}/{file_name}"
    if quiet:
        command = f"wget -q -O {chromedriver_zip} {url}"
    else:
        command = f"wget -O {chromedriver_zip} {url}"
    print(f"Downloading: {file_name}")
    # os.system(command)
    !$command

    # Extract chromedriver from zip
    with zipfile.ZipFile(chromedriver_zip) as zpf:
        zpf.extract(member="chromedriver-linux64/chromedriver", path=work_dir)

    # Remove chromedriver-linux64.zip file
    os.remove(chromedriver_zip)

    # Move extracted chromedriver binary file to /usr/bin directory
    source = f"{work_dir}/chromedriver-linux64/chromedriver"
    destination = "/usr/bin/chromedriver"
    os.rename(source, destination)

    # Make chromedriver binary executable
    os.system(f"chmod +x {destination}")

    # Remove empty chromedriver-linux64 folder
    shutil.rmtree(f"{work_dir}/chromedriver-linux64")

    print("Chromedriver installed")

def install_selenium_package(quiet: bool):
    if quiet:
        !pip install selenium -qq >> pip.log
    else:
        !pip install selenium

def main(quiet: bool):
    # Get the latest version of chromium from linux mint packages site
    latest_version = get_chromium_latest_version()
    # Name of the deb file
    deb_file = urllib.parse.unquote(latest_version, "utf-8")
    # Download and install chromium for ubuntu 22.04
    install_chromium(latest_version, deb_file, quiet)
    # Check if installation succesfull
    check_chromium_installation(deb_file)
    # Install chromedriver
    install_chromedriver(deb_file, quiet)
    # Finally install selenium package
    install_selenium_package(quiet)

if __name__ == '__main__':
    quiet = True # verboseness of wget and apt
    main(quiet)

Downloading: chromium_131.0.6778.85~linuxmint1+virginia_amd64.deb
Installing: chromium_131.0.6778.85~linuxmint1+virginia_amd64.deb
Chromium installation successfull.

Downloading: chromedriver-linux64.zip
Chromedriver installed


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')

chrome_driver_path = 'chromedriver'
driver = webdriver.Chrome(options=options)

url = "https://www.ptt.cc/bbs/index.html"
driver.get(url)
cookies = {"name": "over18", "value": "1"}
driver.add_cookie(cookies)
board_elements = driver.find_elements(By.CSS_SELECTOR, ".b-ent")
top_boards = []


for i in range(3):
    board_link = board_elements[i].find_element(By.CSS_SELECTOR, "a").get_attribute("href")
    board_name = board_link.split('/')[-2]
    top_boards.append((board_name, board_link))

all_articles = []

for board_name, board_link in top_boards:
  for page in range(5):
    current_page_url = board_link if page == 0 else f"{board_link}?page={page+1}"

    driver.get(current_page_url)
    time.sleep(1.5)

    article_elements = driver.find_elements(By.CSS_SELECTOR, ".r-ent")

    for article in article_elements:
        title_element = article.find_element(By.CSS_SELECTOR, ".title")
        title = title_element.text
        link = title_element.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        all_articles.append([board_name, title, link])

for articles in all_articles:
  print(f"新聞類別: {articles[0]}")
  print(f"新聞標題: {articles[1]}")
  print(f"URL網址: {articles[2]}")
  print("-" * 50)

df = pd.DataFrame(all_articles,columns=["新聞類別", "新聞標題", "URL網址"])
df.to_csv("PTT_articles.csv", encoding='utf-8')
print("文章已保存到 熱門看版文章.csv")
driver.quit()

新聞類別: Gossiping
新聞標題: [新聞] 北市某國中遭控消極處理學生持刀恐嚇事件
URL網址: https://www.ptt.cc/bbs/Gossiping/M.1733467965.A.D73.html
--------------------------------------------------
新聞類別: Gossiping
新聞標題: [問卦] 為什麼是年輕人會為國家站出來
URL網址: https://www.ptt.cc/bbs/Gossiping/M.1733467987.A.05A.html
--------------------------------------------------
新聞類別: Gossiping
新聞標題: [問卦] 強烈呼籲周杰倫讓台大IU當演唱會嘉賓
URL網址: https://www.ptt.cc/bbs/Gossiping/M.1733467989.A.01C.html
--------------------------------------------------
新聞類別: Gossiping
新聞標題: [問卦] 彭弋航這樣拿槍指著你Ok嗎？
URL網址: https://www.ptt.cc/bbs/Gossiping/M.1733468012.A.50C.html
--------------------------------------------------
新聞類別: Gossiping
新聞標題: [問卦] 改完排氣管了 要注意什麼
URL網址: https://www.ptt.cc/bbs/Gossiping/M.1733468022.A.489.html
--------------------------------------------------
新聞類別: Gossiping
新聞標題: [問卦] 今天也有柯有倫? 難道[周杰倫]是一個團體
URL網址: https://www.ptt.cc/bbs/Gossiping/M.1733468111.A.5C0.html
--------------------------------------------------
新聞類別: Gossiping
新聞標題: [問卦] 殯儀館或火葬場附設安樂艙
URL網